In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [2]:
#create custom Dataset
encoder = LabelEncoder()
class CustomDataset(Dataset):
    def __init__(self):
        data = pd.read_csv("./dataset/IRIS.csv", sep=",", header=1)
        self.y = encoder.fit_transform(data.iloc[::1,4]) # map 0,1,2 to labels
        self.x = torch.from_numpy(data.iloc[:,:-1].values).float() # classes converted from df to tensor
        self.len = len(data) 
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [3]:
#load dataset and split in training and testset
dataset = CustomDataset()
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [4]:
#define batch size and laod data 
batch_size = 10
train_set = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 
test_set = DataLoader(test_dataset, batch_size=batch_size*2)

In [5]:
#create NN-Network
#4 inputs features and 3 output categories 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack =  nn.Sequential(
         nn.Linear(4,128),
         nn.ReLU(),
         nn.Linear(128,64),
         nn.ReLU(),
         nn.Linear(64,3)
         )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [6]:
# define Parameters and load NN
lr = 0.01
model = NeuralNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
def train_model(train_set, model, optimizer, loss_fn):
    n_epochs = 800
    print_every = n_epochs // 80
    losses = []

    for epoch in range(n_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for i, (inputs, labels) in enumerate(train_set):
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        accuracy = correct_predictions / total_samples
        losses.append(running_loss / len(train_set))

        if epoch % print_every == 0 or epoch == n_epochs - 1:
            print(f"Epoch [{epoch}/{n_epochs}], Loss: {running_loss / len(train_set):.4f}, Accuracy: {accuracy:.4f}")

    #return losses
train_model(train_set, model, optimizer, loss_fn)

Epoch [0/800], Loss: 0.8167, Accuracy: 0.6083
Epoch [80/800], Loss: 0.0922, Accuracy: 0.9500
Epoch [160/800], Loss: 0.0827, Accuracy: 0.9750
Epoch [240/800], Loss: 0.0585, Accuracy: 0.9667
Epoch [320/800], Loss: 0.0605, Accuracy: 0.9833
Epoch [400/800], Loss: 0.0548, Accuracy: 0.9750
Epoch [480/800], Loss: 0.0675, Accuracy: 0.9750
Epoch [560/800], Loss: 0.0626, Accuracy: 0.9750
Epoch [640/800], Loss: 0.0649, Accuracy: 0.9667
Epoch [720/800], Loss: 0.0565, Accuracy: 0.9833
Epoch [799/800], Loss: 0.0556, Accuracy: 0.9750


In [8]:
#Evaluate the Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Y_pred = []
Y_hat = []

with torch.no_grad():
    for i, (inputs, labels) in enumerate(test_set):
        out = model(inputs.float())
        _, out_hat = torch.max(out, 1)
        Y_pred.append(out_hat)
        Y_hat.append(labels)
          
Y_pred = torch.cat(Y_pred)
Y_hat = torch.cat(Y_hat)

# Convert to numpy arrays
y_true = Y_hat.cpu().detach().numpy()
y_pred = Y_pred.cpu().detach().numpy()

# Print classification report with zero_division='warn'
print(classification_report(y_true=y_true, y_pred=y_pred, zero_division='warn'))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        10

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



In [9]:
#Small test witt random datapoint:
output_test = model(torch.tensor([4.5,2.3,1.3,0.3]))
output_test

tensor([ 16.6301,  -6.5155, -44.1537], grad_fn=<ViewBackward0>)

In [10]:
df = pd.read_csv("./dataset/IRIS.csv", sep=",")
y = df.iloc[::1,4]
encoder.fit_transform(y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [11]:
output_test = torch.argmax(output_test)
flower = output_test.item()
encoder.inverse_transform([flower]).item()

'Iris-setosa'